In [11]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [12]:
filename = "ABox.ttl"

In [13]:
text1 = '''CQ_4.1
What are the cultural resources and the time spans of their conception events?
'''

query1 = '''
PREFIX obj: <http://purl.org/changes/object/development/04/schema/>

SELECT ?resource ?beginning ?end
WHERE {
  ?conception_event a obj:ConceptionEvent ;
                    obj:initiates ?resource ;
                    obj:hasTimeInterval ?timeInterval .
  ?timeInterval obj:hasBeginning ?beginning ;
                 obj:hasEnd ?end .
}
'''

In [14]:
text2 = '''CQ_4.2
What are the cultural resources created in a time interval that includes the year 1590?
'''

query2 = '''
PREFIX obj: <http://purl.org/changes/object/development/04/schema/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?resource ?beginning ?end
WHERE {
    ?conception_event a obj:ConceptionEvent ;
                    obj:initiates ?resource ;
                    obj:hasTimeInterval ?timeInterval .
    ?timeInterval obj:hasBeginning ?beginning ;
                        obj:hasEnd ?end .
    
FILTER (xsd:dateTimeTime(concat(str(?beginning), "T00:00:00")) < xsd:dateTimeTime("1591-01-01T00:00:00") && 
xsd:dateTimeTime(concat(str(?end), "T00:00:00")) > xsd:dateTimeTime("1589-12-31T00:00:00"))
}
'''

In [15]:
queries = [(text1, query1),
           (text2, query2)]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))

CQ_4.1
What are the cultural resources and the time spans of their conception events?

+------------------------------------------------------------+-------------+------------+
| resource                                                   | beginning   | end        |
|------------------------------------------------------------+-------------+------------|
| http://purl.org/changes/object/development/04/data/L1-work | 1589-01-01  | 1593-12-31 |
| http://purl.org/changes/object/development/04/data/1-work  | 1482-01-01  | 1482-12-31 |
| http://purl.org/changes/object/development/04/data/26-work | 1500-01-01  | 1599-12-31 |
+------------------------------------------------------------+-------------+------------+
CQ_4.2
What are the cultural resources created in a time interval that includes the year 1590?

+------------------------------------------------------------+-------------+------------+
| resource                                                   | beginning   | end        |
|------